### Read info json file

In [8]:
import json
from pprint import pprint
import numpy as np
import math

def read_json(json_path, video_filename):
    with open(json_path) as data_file:    
        seg = json.load(data_file)

    locs = seg['locations']
    loc2nparray = lambda locs, key: np.array([x[key] for x in locs]).ravel()
    res = {}
    bad_video_c = 0
    bad_video_t = 0
    bad_video_same = 0
    for ifile, f in enumerate(locs):
        if int(f['course']) == -1 or int(f['speed']) == -1:
            bad_video_c += 1   # Changed for interpolation
            if bad_video_c >= 3:
                break
        if ifile != 0:
            if int(f['timestamp'])-prev_t > 1100:
                bad_video_t = 1
                break
            if abs(int(f['timestamp']) - int(prev_t)) < 1:
                bad_video_same = 1
                break
        prev_t = f['timestamp']
    if bad_video_c >= 3:
        print('This is a bad video because course or speed is -1', json_path, video_filename)
        return None
    if bad_video_t:
        print('This is a bad video because time sample not uniform', json_path, video_filename)
        return None
    if len(locs)==0:
        print('This is a bad video because no location data available', json_path, video_filename)
        return None
    if bad_video_same:
        print('This is a bad video because same timestamps', json_path, video_filename)
        return None
    
    for key in locs[0].keys():
        res[key]=loc2nparray(locs, key)

    # add the starting time point and ending time point as well
    res['startTime'] = seg['startTime']
    res['endTime'] = seg['endTime']

    if res['timestamp'][0] - res['startTime'] > 2000:
        print('This is bad video because starting time too far ahead', json_path, video_filename)
        return None

    if res['endTime'] - res['timestamp'][-1] > 2000:
        print('This is bad video because ending time too far ahead', json_path, video_filename)
        return None

    return res

### Filling speed from 40s to 602frames

In [9]:
def fill_missing_speeds_and_courses(values, show_warning):
    l = len(values)
    for i in range(l):
        if values[i] == -1:
            if show_warning:
                print("Warning: course==-1 appears, previous computation might not be reliable")
            if i == (l-1):
                values[i] = values[i-1]
            else:
                if values[i+1] == -1:
                    return None
                values[i] = values[i+1]
    return values

In [10]:
def get_interpolated_speed_xy(res, hz=15):     
    def vec(speed, course):
        t = math.radians(course)
        return np.array([math.sin(t)*speed, math.cos(t)*speed])
    
    course = res['course']
    speed0 = res['speed']
    # first convert to speed vecs
    l=len(course)
    speed = np.zeros((l, 2), dtype = np.float32)
    for i in range(l):
        # interpolate when the number of missing speed is small
        speed0 = fill_missing_speeds_and_courses(speed0, False)
        course = fill_missing_speeds_and_courses(course, True)
        if (speed0 is None) or (course is None):
            return None

        speed[i,:] = vec(speed0[i], course[i])

    tot_ms = res['endTime'] - res['startTime']
    # total number of output
    nout = tot_ms * hz // 1000
    out = np.zeros((nout, 2), dtype=np.float32)
    
    # if time is t second, there should be t+1 points
    last_start = 0
    ts = res['timestamp']
    for i in range(nout):
        # convert to ms timestamp
        timenow = i * 1000.0 / hz + res['startTime']  
        
        while (last_start+1 < len(ts)) and (ts[last_start+1] < timenow):
            last_start += 1
 
        if last_start+1 == len(ts):                    
            out[i, :] = speed[last_start, :]           
        elif timenow <= ts[0]:
            out[i, :] = speed[0, :]
        else:
            time1 = timenow - ts[last_start]
            time2 = ts[last_start+1] - timenow
            r1 = time2 / (time1 + time2)
            r2 = time1 / (time1 + time2)
            inter = r1*speed[last_start, :] + r2*speed[last_start+1, :]
            out[i, :] = inter
    return out

In [11]:
def get_interpolated_speed(json_path, video_filename, hz):
    res = read_json(json_path, video_filename)
    if res is None:
        return None
    out = get_interpolated_speed_xy(res, hz)
    return out

In [13]:
speed_list = get_interpolated_speed('C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val/b1ca2e5d-84cf9134.json',_,15)

### Speed to angle

In [14]:
def speed_to_course(speed):
    pi = math.pi
    if speed[1] == 0:
        if speed[0] > 0:
            course = pi / 2
        elif speed[0] == 0:
            course = None
        elif speed[0] < 0:
            course = 3 * pi / 2
        return course
    course = math.atan(speed[0] / speed[1])
    if course < 0:
        course = course + 2 * pi
    if speed[1] > 0:
        course = course
    else:
        course = pi + course
        if course > 2 * pi:
            course = course - 2 * pi
    assert not math.isnan(course)
    return course

In [15]:
def to_course_list(speed_list):
        l = speed_list.shape[0]
        course_list = []
        for i in range(l):
            speed = speed_list[i,:]
            course_list.append(speed_to_course(speed))
        return course_list

### Turn dict and action

In [16]:
turn_str2int={'not_sure': -1, 'straight': 0, 'slow_or_stop': 1,
                  'turn_left': 2, 'turn_right': 3,
                  'turn_left_slight': 4, 'turn_right_slight': 5,}
                  #'acceleration': 6, 'deceleration': 7}

turn_int2str={y: x for x, y in turn_str2int.items()}
naction = np.sum(np.less_equal(0, np.array([-1, 0, 1, 2, 3, 4, 5])))

In [17]:
def turning_heuristics(speed_list, speed_limit_as_stop=0, no_slight_turn=False, deceleration_thres=1.0 ):
        if speed_list is None:
            return None
        
        course_list = to_course_list(speed_list)
        speed_v = np.linalg.norm(speed_list, axis=1)
        l = len(course_list)
        action = np.zeros(l).astype(np.int32)
        course_diff = np.zeros(l).astype(np.float32)

        enum = turn_str2int

        thresh_low = (2*math.pi / 360)*1
        thresh_high = (2*math.pi / 360)*35
        thresh_slight_low = (2*math.pi / 360)*3

        def diff(a, b):
            # return a-b \in -pi to pi
            d = a - b
            if d > math.pi:
                d -= math.pi * 2
            if d < -math.pi:
                d += math.pi * 2
            return d

        for i in range(l):
            if i == 0:
                action[i] = enum['not_sure']
                continue

            # the speed_limit_as_stop should be small,
            # this detect strict real stop
            if speed_v[i] < speed_limit_as_stop + 1e-3:
                # take the smaller speed as stop
                action[i] = enum['slow_or_stop']
                continue

            course = course_list[i]
            prev = course_list[i-1]

            if course is None or prev is None:
                action[i] = enum['slow_or_stop']
                course_diff[i] = 9999
                continue

            course_diff[i] = diff(course, prev)*360/(2*math.pi)
            if thresh_high > diff(course, prev) > thresh_low:
                if diff(course, prev) > thresh_slight_low:
                    action[i] = enum['turn_right']
                else:
                    action[i] = enum['turn_right_slight']

            elif -thresh_high < diff(course, prev) < -thresh_low:
                if diff(course, prev) < -thresh_slight_low:
                    action[i] = enum['turn_left']
                else:
                    action[i] = enum['turn_left_slight']
            elif diff(course, prev) >= thresh_high or diff(course, prev) <= -thresh_high:
                action[i] = enum['not_sure']
            else:
                action[i] = enum['straight']

            if no_slight_turn:
                if action[i] == enum['turn_left_slight']:
                    action[i] = enum['turn_left']
                if action[i] == enum['turn_right_slight']:
                    action[i] = enum['turn_right']

            # this detect significant slow down that is not due to going to turn
            if deceleration_thres > 0 and action[i] == enum['straight']:
                hz = 15 #FLAGS.frame_rate / FLAGS.temporal_downsample_factor
                acc_now = (speed_v[i] - speed_v[i - 1]) / (1.0/hz)
                if acc_now < - deceleration_thres:
                    action[i] = enum['slow_or_stop']
                    continue

        # avoid the initial uncertainty
        action[0] = action[1]
        return action

In [19]:
turning_heuristics(speed_list, speed_limit_as_stop=0, no_slight_turn=True, deceleration_thres=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### action for val

In [20]:
val_list = [line.rstrip('\n') for line in open('./val_id.txt')]

In [36]:
def info2action(val_list,info_path):

    import os
    val_info_list = []
    for val_name in val_list:
        val_info = {'id':0,'action':0}
        file_path = os.path.join(info_path,os.path.splitext(val_name)[0]+'.json')
        val_info['id'] = os.path.splitext(val_name)[0]
        speed_list = get_interpolated_speed(file_path,_,15)
        action_list = turning_heuristics(speed_list, speed_limit_as_stop=0, no_slight_turn=True, deceleration_thres=1.0)
        
        if action_list is None:
            val_info['action'] = 'N/A'
        else:
            val_info['action'] = action_list
        val_info_list.append(val_info)
    return val_info_list
    

In [22]:
info_path = 'C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val'

In [37]:
val_info_list = info2action(val_list,info_path)

This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val\b255cd6c-f2f28aff.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val\b2e54795-db1f3bad.json 743
This is a bad video because no location data available C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val\b3091a27-aa7841b5.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val\b3853f81-272f6c37.json 743
This is a bad video because no location data available C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val\b48fc70b-9d91e215.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val\b4a8ec73-893c4da4.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/val\b57dee9d-e5bd3

In [38]:
len(val_info_list)

743

In [39]:
with open('val_info.txt', 'w') as f:
    for item in val_info_list:
        f.write("%s\n" % item)

### action for train

In [40]:
train_list = [line.rstrip('\n') for line in open('./train_id.txt')]

In [41]:
info_path = 'C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train'

In [42]:
train_info_list = info2action(train_list,info_path)

This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\002f8552-0cdd55c6.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\00690c26-e4bbbd72.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\02ac8427-bd3b76a5.json 743
This is a bad video because time sample not uniform C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\034daca2-a038e325.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\051d857c-faeca4ad.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\064ecbd3-e42226ee.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\066e4bec-ba1

This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\2c02054a-65126d9c.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\2d930518-32637aa5.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\2d9aefec-2927f6af.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\2e4af06d-4bf647b7.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\2efc2cb4-92513adc.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\2f81fa71-f3ce4c09.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\2fe8c6a4-66b40

This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\5096d11c-058ed0c9.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\5224f0b3-57a70fe3.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\5262ac9b-1e7c910d.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\52d25abb-afaee24b.json 743
This is a bad video because time sample not uniform C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\536b8e93-ad8f066b.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\54ea1a40-83900c32.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\5654d8db-83c

This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\73ff1213-ee5ffa36.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\74308a03-3216714e.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\746a720a-dfe93574.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\74813bcb-5dd1a6db.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\74813bcb-70eec853.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\74e4c6c9-365138c8.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\752565d6-cd29e

This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\92a79979-fd0da09d.json 743
This is a bad video because time sample not uniform C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\93820c7b-84e176d2.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\952ae034-b3ac41a6.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\95751d19-9a77896e.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\95bd3f58-5e955b7c.json 743
This is a bad video because course or speed is -1 C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\95e454b2-f4aa367d.json 743
This is bad video because starting time too far ahead C:/Users/SelfDriving/Desktop/bdd100k_info/bdd100k/info/100k/train\95eb

In [43]:
len(train_info_list)

5070

In [44]:
with open('train_info.txt', 'w') as f:
    for item in train_info_list:
        f.write("%s\n" % item)